# Goal:
Create our NBA career outcome probabilities for every 2020 NBA Draft prospect.

Import libraries

In [14]:
! pip install joblib tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import itertools
import joblib as jb
from sklearn.preprocessing import scale, LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
sns.set_style('whitegrid')
sns.set(rc = {'figure.figsize':(12, 8)})

Read in our models and their required features.

In [15]:
first_features = jb.load('../Models/nba_model_kbest.pkl')
first_model = jb.load('../Models/nba_model.pkl')
second_model = tf.keras.models.load_model('../Models/active_players_career_outlook_model.h5')
second_model_inputs = jb.load('../Models/second_model_inputs.pkl')

Read in our 2020 NBA Draft class prospect data.

In [16]:
active_data = pd.read_csv('../Data/active_binned.csv')
active_data.head()

,ncaa_id,player_name,nba_career_quality,made_nba,field_goals_bin_1,field_goals_bin_2,field_goals_bin_3,field_goals_bin_4,three_point_attempt_rate_bin_1,three_point_attempt_rate_bin_2,three_point_attempt_rate_bin_3,defensive_win_shares_bin_1,defensive_win_shares_bin_2,defensive_win_shares_bin_3,defensive_win_shares_bin_4,blocks_bin_1,blocks_bin_2,blocks_bin_3,blocks_bin_4,blocks_bin_5,games_played_bin_1,games_played_bin_2,games_played_bin_3,turnovers_bin_1,turnovers_bin_2,turnovers_bin_3,turnovers_bin_4,turnover_percentage_bin_1,turnover_percentage_bin_2,turnover_percentage_bin_3,total_rebounds_bin_1,total_rebounds_bin_2,total_rebounds_bin_3,total_rebounds_bin_4,three_pointers_bin_1,three_pointers_bin_2,three_pointers_bin_3,three_pointers_bin_4,games_started_bin_1,games_started_bin_2,games_started_bin_3,games_started_bin_4,three_point_percentage_bin_1,three_point_percentage_bin_2,three_point_percentage_bin_3,three_point_percentage_bin_4,offensive_win_shares_bin_1,offensive_win_shares_bin_2,offensive_win_shares_bin_3,weight_bin_1,weight_bin_2,weight_bin_3,weight_bin_4,weight_bin_5,free_throw_percentage_bin_1,free_throw_percentage_bin_2,free_throw_percentage_bin_3,two_pointers_bin_1,two_pointers_bin_2,two_pointers_bin_3,two_pointers_bin_4,high_school_national_rank_bin_1,high_school_national_rank_bin_2,defensive_rebounds_bin_1,defensive_rebounds_bin_2,defensive_rebounds_bin_3,defensive_rebounds_bin_4,free_throw_attempts_bin_1,free_throw_attempts_bin_2,free_throw_attempts_bin_3,free_throw_attempts_bin_4,field_goal_percentage_bin_1,field_goal_percentage_bin_2,field_goal_percentage_bin_3,field_goal_percentage_bin_4,field_goal_percentage_bin_5,two_point_attempts_bin_1,two_point_attempts_bin_2,two_point_attempts_bin_3,two_point_attempts_bin_4,field_goal_attempts_bin_1,field_goal_attempts_bin_2,field_goal_attempts_bin_3,field_goal_attempts_bin_4,two_point_percentage_bin_1,two_point_percentage_bin_2,two_point_percentage_bin_3,two_point_percentage_bin_4,steals_bin_1,steals_bin_2,steals_bin_3,steals_bin_4,free_throw_attempt_rate_bin_1,free_throw_attempt_rate_bin_2,free_throw_attempt_rate_bin_3,free_throw_attempt_rate_bin_4,free_throw_attempt_rate_bin_5,free_throw_attempt_rate_bin_6,points_bin_1,points_bin_2,points_bin_3,points_bin_4,usage_percentage_bin_1,usage_percentage_bin_2,usage_percentage_bin_3,three_point_attempts_bin_1,three_point_attempts_bin_2,three_point_attempts_bin_3,three_point_attempts_bin_4,offensive_rebounds_bin_1,offensive_rebounds_bin_2,offensive_rebounds_bin_3,offensive_rebounds_bin_4,high_school_recruit_score_bin_2,high_school_recruit_score_bin_3,high_school_recruit_score_bin_4,high_school_recruit_score_bin_5,high_school_recruit_score_bin_6,draft_rank_bin_1,draft_rank_bin_2,draft_rank_bin_3,draft_rank_bin_4,draft_rank_bin_5,assists_bin_1,assists_bin_2,assists_bin_3,assists_bin_4,personal_fouls_bin_1,personal_fouls_bin_2,personal_fouls_bin_3,personal_fouls_bin_4,assist_percentage_bin_1,assist_percentage_bin_2,assist_percentage_bin_3,assist_percentage_bin_4,minutes_played_bin_1,minutes_played_bin_2,minutes_played_bin_3,minutes_played_bin_4,height_bin_1,height_bin_2,height_bin_3,height_bin_4,height_bin_5,made_nba_bin_1,made_nba_bin_2,height_to_weight_bin_1,height_to_weight_bin_2,height_to_weight_bin_3,height_to_weight_bin_4,height_to_weight_bin_6,weight_to_height_bin_1,weight_to_height_bin_2,weight_to_height_bin_3,weight_to_height_bin_4,weight_to_height_bin_5,weight_to_height_bin_6,recruit_draft_rank_mean_bin_1,recruit_draft_rank_mean_bin_2,draft_recruit_ratio_bin_1,draft_recruit_ratio_bin_2,draft_recruit_ratio_bin_3,draft_recruit_ratio_bin_4,assist_to_turnover_ratio_bin_1,assist_to_turnover_ratio_bin_2,assist_to_turnover_ratio_bin_3,assist_to_turnover_ratio_bin_4,assist_to_turnover_ratio_bin_5,assist_to_turnover_ratio_bin_6,assists_per_minute_bin_1,assists_per_minute_bin_2,assists_per_minute_bin_3,assists_per_minute_bin_4,assists_per_minute_bin_5,assists_per_minute_bin_6,blocks_per_minute_bin_1,blocks_per_minute_bin_2,bl

In [17]:
active_data.head()

,ncaa_id,player_name,nba_career_quality,made_nba,field_goals_bin_1,field_goals_bin_2,field_goals_bin_3,field_goals_bin_4,three_point_attempt_rate_bin_1,three_point_attempt_rate_bin_2,three_point_attempt_rate_bin_3,defensive_win_shares_bin_1,defensive_win_shares_bin_2,defensive_win_shares_bin_3,defensive_win_shares_bin_4,blocks_bin_1,blocks_bin_2,blocks_bin_3,blocks_bin_4,blocks_bin_5,games_played_bin_1,games_played_bin_2,games_played_bin_3,turnovers_bin_1,turnovers_bin_2,turnovers_bin_3,turnovers_bin_4,turnover_percentage_bin_1,turnover_percentage_bin_2,turnover_percentage_bin_3,total_rebounds_bin_1,total_rebounds_bin_2,total_rebounds_bin_3,total_rebounds_bin_4,three_pointers_bin_1,three_pointers_bin_2,three_pointers_bin_3,three_pointers_bin_4,games_started_bin_1,games_started_bin_2,games_started_bin_3,games_started_bin_4,three_point_percentage_bin_1,three_point_percentage_bin_2,three_point_percentage_bin_3,three_point_percentage_bin_4,offensive_win_shares_bin_1,offensive_win_shares_bin_2,offensive_win_shares_bin_3,weight_bin_1,weight_bin_2,weight_bin_3,weight_bin_4,weight_bin_5,free_throw_percentage_bin_1,free_throw_percentage_bin_2,free_throw_percentage_bin_3,two_pointers_bin_1,two_pointers_bin_2,two_pointers_bin_3,two_pointers_bin_4,high_school_national_rank_bin_1,high_school_national_rank_bin_2,defensive_rebounds_bin_1,defensive_rebounds_bin_2,defensive_rebounds_bin_3,defensive_rebounds_bin_4,free_throw_attempts_bin_1,free_throw_attempts_bin_2,free_throw_attempts_bin_3,free_throw_attempts_bin_4,field_goal_percentage_bin_1,field_goal_percentage_bin_2,field_goal_percentage_bin_3,field_goal_percentage_bin_4,field_goal_percentage_bin_5,two_point_attempts_bin_1,two_point_attempts_bin_2,two_point_attempts_bin_3,two_point_attempts_bin_4,field_goal_attempts_bin_1,field_goal_attempts_bin_2,field_goal_attempts_bin_3,field_goal_attempts_bin_4,two_point_percentage_bin_1,two_point_percentage_bin_2,two_point_percentage_bin_3,two_point_percentage_bin_4,steals_bin_1,steals_bin_2,steals_bin_3,steals_bin_4,free_throw_attempt_rate_bin_1,free_throw_attempt_rate_bin_2,free_throw_attempt_rate_bin_3,free_throw_attempt_rate_bin_4,free_throw_attempt_rate_bin_5,free_throw_attempt_rate_bin_6,points_bin_1,points_bin_2,points_bin_3,points_bin_4,usage_percentage_bin_1,usage_percentage_bin_2,usage_percentage_bin_3,three_point_attempts_bin_1,three_point_attempts_bin_2,three_point_attempts_bin_3,three_point_attempts_bin_4,offensive_rebounds_bin_1,offensive_rebounds_bin_2,offensive_rebounds_bin_3,offensive_rebounds_bin_4,high_school_recruit_score_bin_2,high_school_recruit_score_bin_3,high_school_recruit_score_bin_4,high_school_recruit_score_bin_5,high_school_recruit_score_bin_6,draft_rank_bin_1,draft_rank_bin_2,draft_rank_bin_3,draft_rank_bin_4,draft_rank_bin_5,assists_bin_1,assists_bin_2,assists_bin_3,assists_bin_4,personal_fouls_bin_1,personal_fouls_bin_2,personal_fouls_bin_3,personal_fouls_bin_4,assist_percentage_bin_1,assist_percentage_bin_2,assist_percentage_bin_3,assist_percentage_bin_4,minutes_played_bin_1,minutes_played_bin_2,minutes_played_bin_3,minutes_played_bin_4,height_bin_1,height_bin_2,height_bin_3,height_bin_4,height_bin_5,made_nba_bin_1,made_nba_bin_2,height_to_weight_bin_1,height_to_weight_bin_2,height_to_weight_bin_3,height_to_weight_bin_4,height_to_weight_bin_6,weight_to_height_bin_1,weight_to_height_bin_2,weight_to_height_bin_3,weight_to_height_bin_4,weight_to_height_bin_5,weight_to_height_bin_6,recruit_draft_rank_mean_bin_1,recruit_draft_rank_mean_bin_2,draft_recruit_ratio_bin_1,draft_recruit_ratio_bin_2,draft_recruit_ratio_bin_3,draft_recruit_ratio_bin_4,assist_to_turnover_ratio_bin_1,assist_to_turnover_ratio_bin_2,assist_to_turnover_ratio_bin_3,assist_to_turnover_ratio_bin_4,assist_to_turnover_ratio_bin_5,assist_to_turnover_ratio_bin_6,assists_per_minute_bin_1,assists_per_minute_bin_2,assists_per_minute_bin_3,assists_per_minute_bin_4,assists_per_minute_bin_5,assists_per_minute_bin_6,blocks_per_minute_bin_1,blocks_per_minute_bin_2,bl

List our first model's features (NBA Draft probability)

In [18]:
first_features = active_data.columns.values[first_features.get_support(True)]
first_features

array(['offensive_rebounds_bin_4', 'high_school_recruit_score_bin_3',
       'high_school_recruit_score_bin_4', 'draft_rank_bin_1',
       'height_bin_2', 'height_bin_3', 'win_shares_per_minute_bin_4',
       'usage_percentage', 'personal_fouls', 'height'], dtype=object)

In [20]:
active_data['nba_pred'] = [x[1] for x in first_model.predict_proba(active_data[first_features])]

In [21]:
roles_dict = {0: 'Benchwarmer',
 1: 'Non-NBA',
 2: 'Replacement Player',
 3: 'Reserve',
 4: 'Role Player',
 5: 'Star',
 6: 'Starter'}

Assign the most likely career role to the `predicted_career_role` column. 

In [22]:
active_data['predicted_career_role'] = [roles_dict[x] for x in [np.argmax(x) for x in second_model.predict_proba(np.array(active_data[second_model_inputs]))]]

Assign every career role probability to its own column.

In [23]:
active_data['benchwarmer_prob'] = None
active_data['non_nba_prob'] = None
active_data['replacement_player_prob'] = None
active_data['reserve_player_prob'] = None
active_data['role_player_prob'] = None
active_data['star_player_prob'] = None
active_data['starter_player_prob'] = None
active_data[['benchwarmer_prob', 'non_nba_prob', 'replacement_player_prob', 'reserve_player_prob', 'role_player_prob', 'star_player_prob', 'starter_player_prob']] = second_model.predict_proba(np.array(active_data[second_model_inputs])) 

Look at our results for top players (Anthony Edwards, Cole Anthony, Nico Mannion).

In [24]:
active_data[active_data['player_name'].isin(['Anthony Edwards', 'Cole Anthony', 'Nico Mannion'])]

,ncaa_id,player_name,nba_career_quality,made_nba,field_goals_bin_1,field_goals_bin_2,field_goals_bin_3,field_goals_bin_4,three_point_attempt_rate_bin_1,three_point_attempt_rate_bin_2,three_point_attempt_rate_bin_3,defensive_win_shares_bin_1,defensive_win_shares_bin_2,defensive_win_shares_bin_3,defensive_win_shares_bin_4,blocks_bin_1,blocks_bin_2,blocks_bin_3,blocks_bin_4,blocks_bin_5,games_played_bin_1,games_played_bin_2,games_played_bin_3,turnovers_bin_1,turnovers_bin_2,turnovers_bin_3,turnovers_bin_4,turnover_percentage_bin_1,turnover_percentage_bin_2,turnover_percentage_bin_3,total_rebounds_bin_1,total_rebounds_bin_2,total_rebounds_bin_3,total_rebounds_bin_4,three_pointers_bin_1,three_pointers_bin_2,three_pointers_bin_3,three_pointers_bin_4,games_started_bin_1,games_started_bin_2,games_started_bin_3,games_started_bin_4,three_point_percentage_bin_1,three_point_percentage_bin_2,three_point_percentage_bin_3,three_point_percentage_bin_4,offensive_win_shares_bin_1,offensive_win_shares_bin_2,offensive_win_shares_bin_3,weight_bin_1,weight_bin_2,weight_bin_3,weight_bin_4,weight_bin_5,free_throw_percentage_bin_1,free_throw_percentage_bin_2,free_throw_percentage_bin_3,two_pointers_bin_1,two_pointers_bin_2,two_pointers_bin_3,two_pointers_bin_4,high_school_national_rank_bin_1,high_school_national_rank_bin_2,defensive_rebounds_bin_1,defensive_rebounds_bin_2,defensive_rebounds_bin_3,defensive_rebounds_bin_4,free_throw_attempts_bin_1,free_throw_attempts_bin_2,free_throw_attempts_bin_3,free_throw_attempts_bin_4,field_goal_percentage_bin_1,field_goal_percentage_bin_2,field_goal_percentage_bin_3,field_goal_percentage_bin_4,field_goal_percentage_bin_5,two_point_attempts_bin_1,two_point_attempts_bin_2,two_point_attempts_bin_3,two_point_attempts_bin_4,field_goal_attempts_bin_1,field_goal_attempts_bin_2,field_goal_attempts_bin_3,field_goal_attempts_bin_4,two_point_percentage_bin_1,two_point_percentage_bin_2,two_point_percentage_bin_3,two_point_percentage_bin_4,steals_bin_1,steals_bin_2,steals_bin_3,steals_bin_4,free_throw_attempt_rate_bin_1,free_throw_attempt_rate_bin_2,free_throw_attempt_rate_bin_3,free_throw_attempt_rate_bin_4,free_throw_attempt_rate_bin_5,free_throw_attempt_rate_bin_6,points_bin_1,points_bin_2,points_bin_3,points_bin_4,usage_percentage_bin_1,usage_percentage_bin_2,usage_percentage_bin_3,three_point_attempts_bin_1,three_point_attempts_bin_2,three_point_attempts_bin_3,three_point_attempts_bin_4,offensive_rebounds_bin_1,offensive_rebounds_bin_2,offensive_rebounds_bin_3,offensive_rebounds_bin_4,high_school_recruit_score_bin_2,high_school_recruit_score_bin_3,high_school_recruit_score_bin_4,high_school_recruit_score_bin_5,high_school_recruit_score_bin_6,draft_rank_bin_1,draft_rank_bin_2,draft_rank_bin_3,draft_rank_bin_4,draft_rank_bin_5,assists_bin_1,assists_bin_2,assists_bin_3,assists_bin_4,personal_fouls_bin_1,personal_fouls_bin_2,personal_fouls_bin_3,personal_fouls_bin_4,assist_percentage_bin_1,assist_percentage_bin_2,assist_percentage_bin_3,assist_percentage_bin_4,minutes_played_bin_1,minutes_played_bin_2,minutes_played_bin_3,minutes_played_bin_4,height_bin_1,height_bin_2,height_bin_3,height_bin_4,height_bin_5,made_nba_bin_1,made_nba_bin_2,height_to_weight_bin_1,height_to_weight_bin_2,height_to_weight_bin_3,height_to_weight_bin_4,height_to_weight_bin_6,weight_to_height_bin_1,weight_to_height_bin_2,weight_to_height_bin_3,weight_to_height_bin_4,weight_to_height_bin_5,weight_to_height_bin_6,recruit_draft_rank_mean_bin_1,recruit_draft_rank_mean_bin_2,draft_recruit_ratio_bin_1,draft_recruit_ratio_bin_2,draft_recruit_ratio_bin_3,draft_recruit_ratio_bin_4,assist_to_turnover_ratio_bin_1,assist_to_turnover_ratio_bin_2,assist_to_turnover_ratio_bin_3,assist_to_turnover_ratio_bin_4,assist_to_turnover_ratio_bin_5,assist_to_turnover_ratio_bin_6,assists_per_minute_bin_1,assists_per_minute_bin_2,assists_per_minute_bin_3,assists_per_minute_bin_4,assists_per_minute_bin_5,assists_per_minute_bin_6,blocks_per_minute_bin_1,blocks_per_minute_bin_2,bl

In [25]:
active_data_csv = active_data[['ncaa_id', 'player_name', 'nba_pred', 'predicted_career_role', 'benchwarmer_prob', 'non_nba_prob', 'replacement_player_prob', 'reserve_player_prob', 'role_player_prob', 'star_player_prob', 'starter_player_prob']]

Dump our data into a csv so we can read it back in when we swap containers.

In [26]:
active_data_csv.to_csv('../Data/active_data_predictions.csv', index = False)

Switch to `core` container in order to insert our results into the database

In [1]:
import getpass
import pandas as pd
import psycopg2

Connect to our database.

In [2]:
mypasswd = getpass.getpass()
conn = psycopg2.connect(database = 'cs20_group4',
                              user = 'fhfrf',#replace with pawprint
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)
cursor = conn.cursor()

········


In [3]:
insert_data = pd.read_csv('../Data/active_data_predictions.csv')
insert_data.head()

,ncaa_id,player_name,nba_pred,predicted_career_role,benchwarmer_prob,non_nba_prob,replacement_player_prob,reserve_player_prob,role_player_prob,star_player_prob,starter_player_prob
0,iran-bennett-1,Iran Bennett,1.000000,Non-NBA,8.125922e-08,5.543595e-01,3.918272e-01,2.564247e-04,8.213609e-03,8.715020e-07,4.534241e-02
1,jakub-dombek-1,Jakub Dombek,0.998781,Replacement Player,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,shakur-juiston-1,Shakur Juiston,1.000000,Reserve,2.000015e-11,3.197494e-01,3.438615e-22,6.801035e-01,7.102528e-18,2.260053e-12,1.470853e-04
3,maxwell-starwood-1,Maxwell Starwood,1.000000,Non-NBA,1.346084e-13,9.999917e-01,5.807572e-14,8.159463e-06,2.181801e-07,1.783722e-11,5.009856e-08
4,james-wiseman-1,James Wiseman,1.000000,Star,1.126775e-07,1.482664e-15,1.548839e-10,1.739439e-09,2.910505e-07,9.999994e-01,1.863512e-07


Define our `active_player_predictions` table schema.

In [4]:
cursor = conn.cursor()
c_table = []
for c in insert_data.columns.tolist():
    if insert_data[c].dtype in [float, int, 'int64']:
        c_table.append('{} numeric'.format(c))
    elif insert_data[c].dtype == 'datetime64[ns]':
        c_table.append('{} TIMESTAMP'.format(c))
    else:
        c_table.append('{} varchar({})'.format(c, max([len(str(x)) for x in insert_data[c] if x is not None])))
create_table = 'DROP TABLE IF EXISTS active_player_predictions; '
create_table += 'CREATE TABLE active_player_predictions ({})'.format(', '.join(c for c in c_table))
cursor.execute(create_table)
conn.commit()

Insert our draft class predictions into the `active_player_predictions` table.

In [5]:
df = insert_data.where(pd.notnull(insert_data), None)
cursor = conn.cursor()
INSERT_SQL = 'INSERT INTO active_player_predictions'
INSERT_SQL += '({}) VALUES'.format(', '.join([x for x in df.columns]))
INSERT_SQL += '({})'.format(''.join(['%s,' * len(df.columns)])[:-1])
with conn, conn.cursor() as cursor:
    for row in df.itertuples(index=False, name=None):
        cursor.execute(INSERT_SQL, row)